In [1]:
# Carga de datos

import pandas as pd
df1 = pd.read_csv('./data/cierres_diarios_L1.csv', parse_dates=['Date'], index_col='Date')
df2 = pd.read_csv('./data/cierres_diarios_L2.csv', parse_dates=['Date'], index_col='Date')
df3 = pd.read_csv('./data/cierres_diarios_L3.csv', parse_dates=['Date'], index_col='Date')

# Preparación de los datos

# Eliminación de nulos
df1.ffill(inplace=True) # rellena con el ultimo precio conocido
df1.bfill(inplace=True) # por si hay alguna celda con NaN al principio
df2.ffill(inplace=True) 
df2.bfill(inplace=True) 
df3.ffill(inplace=True) 
df3.bfill(inplace=True)


import pandas as pd
import numpy as np
import tensorflow as tf
from joblib import load
from typing import List

# Cargar scaler y columnas base (mismo orden que en entrenamiento)
scaler = load("scaler_modelos.joblib")


# Como el scaler tiene 'feature_names_in_', usamos ese orden:
feature_order = list(scaler.feature_names_in_)


# Filtramos/ordenamos exactamente como en entrenamiento
df1 = df1[feature_order].copy()
df2 = df2[feature_order].copy()
df3 = df3[feature_order].copy()


# Escalar con el mismo scaler
df1_scaled = pd.DataFrame(
    scaler.transform(df1[feature_order]),
    index=df1.index,
    columns=feature_order
).astype("float32")

df2_scaled = pd.DataFrame(
    scaler.transform(df2[feature_order]),
    index=df2.index,
    columns=feature_order
).astype("float32")

df3_scaled = pd.DataFrame(
    scaler.transform(df3[feature_order]),
    index=df3.index,
    columns=feature_order
).astype("float32")

# === Datasets coherentes con el entrenamiento: usar SIEMPRE datos ESCALADOS ===
def make_dataset(data_scaled, window_size, horizon, batch_size=32, shuffle=False):
    ds = tf.keras.preprocessing.timeseries_dataset_from_array(
        data=data_scaled.values,
        targets=None,
        sequence_length=window_size + horizon,
        sequence_stride=1,
        batch_size=batch_size,
        shuffle=shuffle
    )
    return ds.map(
        lambda seq: (
            tf.cast(seq[:, :window_size, :], tf.float32),   # X
            tf.cast(seq[:, window_size:, :], tf.float32)    # y
        )
    )

window_size = 120
horizon = 20

# Evaluar SOLO el primer (120->29) del intervalo:
ds1 = make_dataset(df1_scaled.iloc[:window_size+horizon], window_size, horizon, batch_size=1, shuffle=False)
ds2 = make_dataset(df2_scaled.iloc[:window_size+horizon], window_size, horizon, batch_size=1, shuffle=False)
ds3 = make_dataset(df3_scaled.iloc[:window_size+horizon], window_size, horizon, batch_size=1, shuffle=False)


2025-09-26 22:56:45.134182: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-09-26 22:56:45.134199: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-09-26 22:56:45.134206: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-09-26 22:56:45.134220: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-09-26 22:56:45.134228: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [2]:
# Funciones comunes

def _eps_tag(x: float) -> str:
    return str(x).replace('.', '_')

def log_cosh_metric(y_true, y_pred):
    e = tf.cast(y_pred, tf.float32) - tf.cast(y_true, tf.float32)
    # logcosh(x) = |x| + softplus(-2|x|) - log(2)  → estable y sin overflow
    ae = tf.abs(e)
    return tf.reduce_mean(ae + tf.nn.softplus(-2.0 * ae) - tf.math.log(2.0))
log_cosh_metric.__name__ = "log_cosh"

def make_huber_loss(delta: float):
    base = tf.keras.losses.Huber(delta=float(delta))
    def huber_loss(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32); y_pred = tf.cast(y_pred, tf.float32)
        return base(y_true, y_pred)
    huber_loss.__name__ = f"huber_delta_{_eps_tag(delta)}"
    return huber_loss

def make_within_eps_vector_metric(eps_vec: np.ndarray, tag: str):
    """eps_vec shape (F,) en la MISMA escala que y_true/y_pred (normalizada)."""
    eps_tf = tf.constant(eps_vec.astype(np.float32), dtype=tf.float32)  # (F,)
    def within_eps(y_true, y_pred):
        diff = tf.abs(tf.cast(y_pred, tf.float32) - tf.cast(y_true, tf.float32))  # (B,H,F)
        thr  = eps_tf[tf.newaxis, tf.newaxis, :]                                   # (1,1,F)
        hit  = tf.cast(diff <= thr, tf.float32)
        return tf.reduce_mean(hit)
    within_eps.__name__ = tag
    return within_eps

def build_within_metrics_minmax(scaler, eps_list: List[float], n_features: int):
    """
    MinMaxScaler: eps = porcentaje * data_range_ por feature.
    OJO: y_true/y_pred están normalizados, así que el umbral ya se pasa NORMALIZADO.
    Con MinMax a [0,1], 'porcentaje del rango' == el propio porcentaje.
    Para mantener la semántica, se usa directamente eps_list (0.5%, 1%, ... del rango).
    """
    metrics = [log_cosh_metric]

    # Verificación rápida de consistencia del scaler
    if not hasattr(scaler, "data_range_"):
        raise ValueError("Se esperaba un MinMaxScaler con atributo data_range_.")
    if len(scaler.data_range_) != n_features:
        raise ValueError("scaler.data_range_ no coincide con n_features.")

    # En escala normalizada [0,1], el 'porcentaje del rango' es exactamente el valor eps.
    # Por lo tanto, el vector de tolerancias NORMALIZADO es uniforme por cada feature (= eps).
    for e in eps_list:
        eps_vec = np.full((n_features,), float(e), dtype=np.float32)
        tag = f"within_eps_{_eps_tag(e)}"
        metrics.append(make_within_eps_vector_metric(eps_vec, tag))
    return metrics

def compute_autc_from_results(res: dict, eps_list: List[float]) -> float:
    """
    Calcula AUTC normalizando por el rango [ε_min, ε_max], ∈ [0,1].
    Toma los valores within_eps_* devueltos por model.evaluate(return_dict=True).
    """
    eps = np.array(sorted(eps_list), dtype=np.float32)
    acc = np.array([res.get(f"within_eps_{str(e).replace('.','_')}", np.nan) for e in eps],
                   dtype=np.float32)

    mask = np.isfinite(acc)
    if mask.sum() < 2:    #Si faltan puntos o hay NaN, integra sobre los disponibles (requiere ≥ 2 puntos).
        return float("nan")

    eps = eps[mask]
    acc = acc[mask]
    return float(np.trapz(acc, eps) / (eps[-1] - eps[0]))


In [3]:
# Carga de modelos
from tensorflow.keras.models import load_model 

# Carga del modelo GRU 
path_gru_s = "./models_gru_huber_sweep/gru_huber_w120_h20_delta0_02793753147_L.keras" 
model_gru_s = load_model(path_gru_s, compile=False) 
delta_gru_s = 0.02793753147125244

# Carga del modelo CNN
path_cnn_s = "./models_cnn_huber_sweep/cnn_huber_w120_h20_delta0_05683806539_L.keras"
model_cnn_s = load_model(path_cnn_s, compile=False)
delta_cnn_s = 0.05683806538581848

# Carga del modelo Transformer
path_trf_s = "./models_transformer_huber_sweep/transformer_huber_w120_h20_delta0_01_L.keras"
delta_trf_s = 0.01

# (Opcional) detectar capas personalizadas si existen en el entorno
def maybe_get_custom_objects():
    names = [
        "PositionalEmbedding", "Custom>PositionalEmbedding",
        "TransformerBlock", "Custom>TransformerBlock",
        # añade aquí otros nombres si los usaste al registrar
    ]
    custom = {}
    for n in names:
        obj = globals().get(n, None)
        if obj is not None:
            custom[n] = obj
    return custom if custom else None

custom_objects = maybe_get_custom_objects()

try:
    if custom_objects:
        model_trf_s = load_model(path_trf_s, compile=False, custom_objects=custom_objects)
    else:
        model_trf_s = load_model(path_trf_s, compile=False)
except Exception as e:
    raise RuntimeError(
        "No se pudo cargar el modelo Transformer. "
        "Si tiene capas personalizadas, hay que pasarlas en custom_objects, por ejemplo:\n"
        "custom={'PositionalEmbedding': PositionalEmbedding, 'TransformerBlock': TransformerBlock}\n"
        f"Error original: {e}"
    )


In [4]:
# === Compilar los modelos con las métricas within-ε correctas (escala normalizada) ===
EPS_LIST = [0.005, 0.01, 0.02, 0.05, 0.1]
n_features = len(feature_order)

metrics = build_within_metrics_minmax(scaler, EPS_LIST, n_features=n_features)
model_gru_s.compile(optimizer="adam", loss=make_huber_loss(delta_gru_s), metrics=metrics)
model_cnn_s.compile(optimizer="adam", loss=make_huber_loss(delta_cnn_s), metrics=metrics)
model_trf_s.compile(optimizer="adam", loss=make_huber_loss(delta_trf_s), metrics=metrics)


def evaluate_and_print(tag, dataset, model_s, name):
    res = model_s.evaluate(dataset, return_dict=True, verbose=0)
    print(f"\n=== Resultados {tag} (120→20) {name} ===")
    print(f"  Huber (loss):              {res.get('loss', float('nan')):.6f}")
    print(f"  log_cosh:                  {res.get('log_cosh', float('nan')):.6f}")
    for e in EPS_LIST:
        key = f"within_eps_{str(e).replace('.','_')}"
        print(f"  {key:26s}: {res.get(key, float('nan')):.6f}")
    autc = compute_autc_from_results(res, EPS_LIST)
    print(f"  AUTC[{min(EPS_LIST):.3f}–{max(EPS_LIST):.3f}]:         {autc:.6f}")
    return res

# === Evaluación de M1, M2 y M3 (datasets completos) ===
res_gru_1 = evaluate_and_print("L1", ds1, model_gru_s, "GRU")
res_gru_2 = evaluate_and_print("L2", ds2, model_gru_s, "GRU")
res_gru_3 = evaluate_and_print("L3", ds3, model_gru_s, "GRU")

res_cnn_1 = evaluate_and_print("L1", ds1, model_cnn_s, "CNN")
res_cnn_2 = evaluate_and_print("L2", ds2, model_cnn_s, "CNN")
res_cnn_3 = evaluate_and_print("L3", ds3, model_cnn_s, "CNN")

res_trf_1 = evaluate_and_print("L1", ds1, model_trf_s, "TRF")
res_trf_2 = evaluate_and_print("L2", ds2, model_trf_s, "TRF")
res_trf_3 = evaluate_and_print("L3", ds3, model_trf_s, "TRF")


# === Ventana ÚNICA (60→5) por cada M para inspección puntual y ver precios reales ===
def single_window_arrays(df_scaled, w=120, h=20):
    if len(df_scaled) < w + h:
        raise ValueError("No hay suficientes días hábiles para 120→20.")
    X = df_scaled.iloc[0:w].to_numpy()[np.newaxis, :, :]              # (1,w,F)
    Y = df_scaled.iloc[w:w+h].to_numpy()[np.newaxis, :, :]            # (1,h,F)
    return X.astype("float32"), Y.astype("float32")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --- 1) Construir tabla con todas las métricas por escenario ---
def res_to_series(tag, res, eps_list):
    d = {
        "loss": res.get("loss", np.nan),
        "log_cosh": res.get("log_cosh", np.nan),
        "AUTC": compute_autc_from_results(res, eps_list),
    }
    for e in eps_list:
        key = f"within_eps_{str(e).replace('.','_')}"
        d[key] = res.get(key, np.nan)
    s = pd.Series(d, name=tag)
    return s

s1_gru = res_to_series("L1", res_gru_1, EPS_LIST)
s2_gru = res_to_series("L2", res_gru_2, EPS_LIST)
s3_gru = res_to_series("L3", res_gru_3, EPS_LIST)

s1_cnn = res_to_series("L1", res_cnn_1, EPS_LIST)
s2_cnn = res_to_series("L2", res_cnn_2, EPS_LIST)
s3_cnn = res_to_series("L3", res_cnn_3, EPS_LIST)

s1_trf = res_to_series("L1", res_trf_1, EPS_LIST)
s2_trf = res_to_series("L2", res_trf_2, EPS_LIST)
s3_trf = res_to_series("L3", res_trf_3, EPS_LIST)


df_all_gru = pd.concat([s1_gru, s2_gru, s3_gru], axis=1).T  # filas: escenarios; cols: métricas
df_mean_gru = df_all_gru.mean(axis=0)

print("\n==== Tabla de métricas por escenario GRU ====")
print(df_all_gru.round(6))
print("\n==== Medias (L1,L2,L3) GRU ====")
print(df_mean_gru.round(6))


df_all_cnn = pd.concat([s1_cnn, s2_cnn, s3_cnn], axis=1).T  # filas: escenarios; cols: métricas
df_mean_cnn = df_all_cnn.mean(axis=0)

print("\n==== Tabla de métricas por escenario CNN ====")
print(df_all_cnn.round(6))
print("\n==== Medias (L1,L2,L3) CNN ====")
print(df_mean_cnn.round(6))


df_all_trf = pd.concat([s1_trf, s2_trf, s3_trf], axis=1).T  # filas: escenarios; cols: métricas
df_mean_trf = df_all_trf.mean(axis=0)

print("\n==== Tabla de métricas por escenario TRF ====")
print(df_all_trf.round(6))
print("\n==== Medias (L1,L2,L3) TRF ====")
print(df_mean_trf.round(6))



2025-09-26 22:56:52.414861: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.



=== Resultados L1 (120→20) GRU ===
  Huber (loss):              0.087425
  log_cosh:                  2.691679
  within_eps_0_005          : 0.000000
  within_eps_0_01           : 0.002000
  within_eps_0_02           : 0.006000
  within_eps_0_05           : 0.028000
  within_eps_0_1            : 0.060000
  AUTC[0.005–0.100]:         0.029000

=== Resultados L2 (120→20) GRU ===
  Huber (loss):              0.094768
  log_cosh:                  2.946241
  within_eps_0_005          : 0.004000
  within_eps_0_01           : 0.006000
  within_eps_0_02           : 0.012000
  within_eps_0_05           : 0.022000
  within_eps_0_1            : 0.040000
  AUTC[0.005–0.100]:         0.022895

=== Resultados L3 (120→20) GRU ===
  Huber (loss):              0.087543
  log_cosh:                  2.676895
  within_eps_0_005          : 0.000000
  within_eps_0_01           : 0.002000
  within_eps_0_02           : 0.004000
  within_eps_0_05           : 0.014000
  within_eps_0_1            : 0.026000
  A

In [5]:
# === Ensemble: promedio simple de (GRU + CNN + TRF) sobre L1, L2, L3 ===
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Average
from tensorflow.keras import Input, Model

# Delta del ensemble como media de los deltas individuales (sólo para el loss Huber de evaluación)
delta_ens_s = float(np.mean([delta_gru_s, delta_cnn_s, delta_trf_s]))

# Construcción del modelo ensemble por promediado simple (mismo input que los modelos base)
inp = Input(shape=model_gru_s.input_shape[1:], name="ensemble_input")
y_gru = model_gru_s(inp, training=False)
y_cnn = model_cnn_s(inp, training=False)
y_trf = model_trf_s(inp, training=False)
y_avg = Average(name="avg_logits")([y_gru, y_cnn, y_trf])
model_ens_s = Model(inp, y_avg, name="ensemble_avg_gru_cnn_trf")

# Compilación con las mismas métricas within-ε y log_cosh que los otros modelos
# (reutilizamos 'metrics' y EPS_LIST ya definidas arriba)
model_ens_s.compile(optimizer="adam", loss=make_huber_loss(delta_ens_s), metrics=metrics)

# Evaluación y salida detallada en el mismo formato
res_ens_1 = evaluate_and_print("L1", ds1, model_ens_s, "ENS(avg)")
res_ens_2 = evaluate_and_print("L2", ds2, model_ens_s, "ENS(avg)")
res_ens_3 = evaluate_and_print("L3", ds3, model_ens_s, "ENS(avg)")

# Conversión de resultados a Series, con AUTC y within-ε, igual que en GRU/CNN/TRF
s1_ens = res_to_series("L1", res_ens_1, EPS_LIST)
s2_ens = res_to_series("L2", res_ens_2, EPS_LIST)
s3_ens = res_to_series("L3", res_ens_3, EPS_LIST)

# DataFrame (filas: escenarios; columnas: métricas)
df_all_ens = pd.concat([s1_ens, s2_ens, s3_ens], axis=1).T
df_mean_ens = df_all_ens.mean(axis=0)

print("\n==== Tabla de métricas por escenario ENS(avg) ====")
print(df_all_ens.round(6))
print("\n==== Medias (L1,L2,L3) ENS(avg) ====")
print(df_mean_ens.round(6))


=== Resultados L1 (120→20) ENS(avg) ===
  Huber (loss):              0.086103
  log_cosh:                  2.411401
  within_eps_0_005          : 0.010000
  within_eps_0_01           : 0.018000
  within_eps_0_02           : 0.030000
  within_eps_0_05           : 0.058000
  within_eps_0_1            : 0.124000
  AUTC[0.005–0.100]:         0.065053

=== Resultados L2 (120→20) ENS(avg) ===
  Huber (loss):              0.093610
  log_cosh:                  2.649162
  within_eps_0_005          : 0.008000
  within_eps_0_01           : 0.014000
  within_eps_0_02           : 0.026000
  within_eps_0_05           : 0.056000
  within_eps_0_1            : 0.128000
  AUTC[0.005–0.100]:         0.064053

=== Resultados L3 (120→20) ENS(avg) ===
  Huber (loss):              0.084452
  log_cosh:                  2.359068
  within_eps_0_005          : 0.000000
  within_eps_0_01           : 0.008000
  within_eps_0_02           : 0.018000
  within_eps_0_05           : 0.048000
  within_eps_0_1           

In [6]:
# === Ensembles ponderados (GRU + CNN + TRF) usando métricas AUTC y log_cosh ===
import numpy as np
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Lambda

# 1) Cálculo de pesos a partir de las métricas agregadas (medias L1,L2,L3) de cada modelo
#    - AUTC: mayor es mejor → pesos ∝ AUTC
#    - log_cosh: menor es mejor → pesos ∝ 1 / log_cosh
def _safe_array(vals):
    vals = np.array(vals, dtype=np.float64)
    if not np.all(np.isfinite(vals)):
        # Si hay NaN/inf, reemplazar por 0 y luego renormalizar
        vals = np.where(np.isfinite(vals), vals, 0.0)
    return vals

def _normalize(w):
    s = np.sum(w)
    if s <= 0 or not np.isfinite(s):
        # fallback a pesos iguales
        return np.array([1/3, 1/3, 1/3], dtype=np.float64)
    return (w / s).astype(np.float64)

# Extraer métricas promedio de cada modelo (ya calculadas arriba)
autc_gru = float(df_mean_gru.get("AUTC", np.nan))
autc_cnn = float(df_mean_cnn.get("AUTC", np.nan))
autc_trf = float(df_mean_trf.get("AUTC", np.nan))

lcsh_gru = float(df_mean_gru.get("log_cosh", np.nan))
lcsh_cnn = float(df_mean_cnn.get("log_cosh", np.nan))
lcsh_trf = float(df_mean_trf.get("log_cosh", np.nan))

# Pesos por AUTC
w_autc_raw = _safe_array([autc_gru, autc_cnn, autc_trf])
w_autc = _normalize(w_autc_raw)

# Pesos por log_cosh inverso (añadimos epsilon para evitar división por 0)
_eps = 1e-12
w_lcsh_raw = _safe_array([1.0 / max(lcsh_gru, _eps),
                          1.0 / max(lcsh_cnn, _eps),
                          1.0 / max(lcsh_trf, _eps)])
w_lcsh = _normalize(w_lcsh_raw)

print("\n== Pesos ENS(w=AUTC) [GRU, CNN, TRF] ==>", np.round(w_autc, 6))
print("== Pesos ENS(w=1/log_cosh) [GRU, CNN, TRF] ==>", np.round(w_lcsh, 6))

# 2) Construcción de los dos ensembles ponderados
inp_w = Input(shape=model_gru_s.input_shape[1:], name="ensemble_w_input")

y_gru_w = model_gru_s(inp_w, training=False)
y_cnn_w = model_cnn_s(inp_w, training=False)
y_trf_w = model_trf_s(inp_w, training=False)

w0a, w1a, w2a = [float(x) for x in w_autc.tolist()]
w0l, w1l, w2l = [float(x) for x in w_lcsh.tolist()]

y_wavg_autc = Lambda(lambda t: t[0]*w0a + t[1]*w1a + t[2]*w2a, name="weighted_avg_autc")([y_gru_w, y_cnn_w, y_trf_w])
y_wavg_lcsh = Lambda(lambda t: t[0]*w0l + t[1]*w1l + t[2]*w2l, name="weighted_avg_lcsh")([y_gru_w, y_cnn_w, y_trf_w])

model_ens_autc_s = Model(inp_w, y_wavg_autc, name="ensemble_w_autc_gru_cnn_trf")
model_ens_lcsh_s = Model(inp_w, y_wavg_lcsh, name="ensemble_w_lcsh_gru_cnn_trf")

# 3) Compilación con mismas métricas; loss Huber con delta medio
delta_ens_s = float(np.mean([delta_gru_s, delta_cnn_s, delta_trf_s]))
model_ens_autc_s.compile(optimizer="adam", loss=make_huber_loss(delta_ens_s), metrics=metrics)
model_ens_lcsh_s.compile(optimizer="adam", loss=make_huber_loss(delta_ens_s), metrics=metrics)

# 4) Evaluación (L1, L2, L3) con el mismo formato
# ENS(w=AUTC)
res_ensa1 = evaluate_and_print("L1", ds1, model_ens_autc_s, "ENS(w=AUTC)")
res_ensa2 = evaluate_and_print("L2", ds2, model_ens_autc_s, "ENS(w=AUTC)")
res_ensa3 = evaluate_and_print("L3", ds3, model_ens_autc_s, "ENS(w=AUTC)")

s1_ensa = res_to_series("L1", res_ensa1, EPS_LIST)
s2_ensa = res_to_series("L2", res_ensa2, EPS_LIST)
s3_ensa = res_to_series("L3", res_ensa3, EPS_LIST)

df_all_ensa = pd.concat([s1_ensa, s2_ensa, s3_ensa], axis=1).T
df_mean_ensa = df_all_ensa.mean(axis=0)

print("\n==== Tabla de métricas por escenario ENS(w=AUTC) ====")
print(df_all_ensa.round(6))
print("\n==== Medias (L1,L2,L3) ENS(w=AUTC) ====")
print(df_mean_ensa.round(6))

# ENS(w=1/log_cosh)
res_ensl1 = evaluate_and_print("L1", ds1, model_ens_lcsh_s, "ENS(w=1/log_cosh)")
res_ensl2 = evaluate_and_print("L2", ds2, model_ens_lcsh_s, "ENS(w=1/log_cosh)")
res_ensl3 = evaluate_and_print("L3", ds3, model_ens_lcsh_s, "ENS(w=1/log_cosh)")

s1_ensl = res_to_series("L1", res_ensl1, EPS_LIST)
s2_ensl = res_to_series("L2", res_ensl2, EPS_LIST)
s3_ensl = res_to_series("L3", res_ensl3, EPS_LIST)

df_all_ensl = pd.concat([s1_ensl, s2_ensl, s3_ensl], axis=1).T
df_mean_ensl = df_all_ensl.mean(axis=0)

print("\n==== Tabla de métricas por escenario ENS(w=1/log_cosh) ====")
print(df_all_ensl.round(6))
print("\n==== Medias (L1,L2,L3) ENS(w=1/log_cosh) ====")
print(df_mean_ensl.round(6))


== Pesos ENS(w=AUTC) [GRU, CNN, TRF] ==> [0.433588 0.232267 0.334145]
== Pesos ENS(w=1/log_cosh) [GRU, CNN, TRF] ==> [0.35934 0.29021 0.35045]

=== Resultados L1 (120→20) ENS(w=AUTC) ===
  Huber (loss):              0.086751
  log_cosh:                  2.427559
  within_eps_0_005          : 0.008000
  within_eps_0_01           : 0.012000
  within_eps_0_02           : 0.022000
  within_eps_0_05           : 0.070000
  within_eps_0_1            : 0.132000
  AUTC[0.005–0.100]:         0.070000

=== Resultados L2 (120→20) ENS(w=AUTC) ===
  Huber (loss):              0.094721
  log_cosh:                  2.674328
  within_eps_0_005          : 0.006000
  within_eps_0_01           : 0.016000
  within_eps_0_02           : 0.030000
  within_eps_0_05           : 0.084000
  within_eps_0_1            : 0.138000
  AUTC[0.005–0.100]:         0.079421

=== Resultados L3 (120→20) ENS(w=AUTC) ===
  Huber (loss):              0.085940
  log_cosh:                  2.391284
  within_eps_0_005          : 

In [7]:
# === Ensembles ponderados mixtos: AUTC vs 1/log_cosh (25/75, 50/50, 75/25) ===
import numpy as np
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Lambda

# Asegurar que existen w_autc y w_lcsh; si no, recomputar desde df_mean_* (medias L1,L2,L3)
def _safe_array(vals):
    vals = np.array(vals, dtype=np.float64)
    vals = np.where(np.isfinite(vals), vals, 0.0)
    return vals

def _normalize(w):
    s = float(np.sum(w))
    if not np.isfinite(s) or s <= 0:
        return np.array([1/3, 1/3, 1/3], dtype=np.float64)
    return (w / s).astype(np.float64)

try:
    w_autc, w_lcsh  # noqa: F401
except NameError:
    autc_gru = float(df_mean_gru.get("AUTC", np.nan))
    autc_cnn = float(df_mean_cnn.get("AUTC", np.nan))
    autc_trf = float(df_mean_trf.get("AUTC", np.nan))
    lcsh_gru = float(df_mean_gru.get("log_cosh", np.nan))
    lcsh_cnn = float(df_mean_cnn.get("log_cosh", np.nan))
    lcsh_trf = float(df_mean_trf.get("log_cosh", np.nan))

    w_autc_raw = _safe_array([autc_gru, autc_cnn, autc_trf])
    w_autc = _normalize(w_autc_raw)

    _eps = 1e-12
    w_lcsh_raw = _safe_array([1.0 / max(lcsh_gru, _eps),
                              1.0 / max(lcsh_cnn, _eps),
                              1.0 / max(lcsh_trf, _eps)])
    w_lcsh = _normalize(w_lcsh_raw)

# Mezclas: alpha para AUTC, (1-alpha) para 1/log_cosh
alphas = [0.25, 0.50, 0.75]
w_mixes = []
for a in alphas:
    w_mix = _normalize(a * w_autc + (1.0 - a) * w_lcsh)
    w_mixes.append(w_mix)

print("\n== Pesos mixtos [GRU, CNN, TRF] ==")
for a, w_mix in zip(alphas, w_mixes):
    print(f"  {int(a*100)}% AUTC + {int((1-a)*100)}% 1/log_cosh ->", np.round(w_mix, 6))

# Construcción de los tres ensembles y evaluación
delta_ens_s = float(np.mean([delta_gru_s, delta_cnn_s, delta_trf_s]))

def build_weighted_ensemble(weights, name_suffix):
    inp = Input(shape=model_gru_s.input_shape[1:], name=f"ens_mix_input_{name_suffix}")
    y0 = model_gru_s(inp, training=False)
    y1 = model_cnn_s(inp, training=False)
    y2 = model_trf_s(inp, training=False)
    w0, w1, w2 = [float(x) for x in weights.tolist()]
    y = Lambda(lambda t: t[0]*w0 + t[1]*w1 + t[2]*w2, name=f"weighted_avg_{name_suffix}")([y0, y1, y2])
    m = Model(inp, y, name=f"ensemble_mix_{name_suffix}")
    m.compile(optimizer="adam", loss=make_huber_loss(delta_ens_s), metrics=metrics)
    return m

models_mix = [
    ("25_75", w_mixes[0], "ENS(25%AUTC+75%1/log_cosh)"),
    ("50_50", w_mixes[1], "ENS(50%AUTC+50%1/log_cosh)"),
    ("75_25", w_mixes[2], "ENS(75%AUTC+25%1/log_cosh)"),
]

# Evaluación y tablas
for tag_mix, w_mix, pretty_name in models_mix:
    model_mix = build_weighted_ensemble(w_mix, tag_mix)
    r1 = evaluate_and_print("L1", ds1, model_mix, pretty_name)
    r2 = evaluate_and_print("L2", ds2, model_mix, pretty_name)
    r3 = evaluate_and_print("L3", ds3, model_mix, pretty_name)

    s1 = res_to_series("L1", r1, EPS_LIST)
    s2 = res_to_series("L2", r2, EPS_LIST)
    s3 = res_to_series("L3", r3, EPS_LIST)

    df_all = pd.concat([s1, s2, s3], axis=1).T
    df_mean = df_all.mean(axis=0)

    print(f"\n==== Tabla de métricas por escenario {pretty_name} ====")
    print(df_all.round(6))
    print(f"\n==== Medias (L1,L2,L3) {pretty_name} ====")
    print(df_mean.round(6))


== Pesos mixtos [GRU, CNN, TRF] ==
  25% AUTC + 75% 1/log_cosh -> [0.377902 0.275725 0.346373]
  50% AUTC + 50% 1/log_cosh -> [0.396464 0.261239 0.342297]
  75% AUTC + 25% 1/log_cosh -> [0.415026 0.246753 0.338221]

=== Resultados L1 (120→20) ENS(25%AUTC+75%1/log_cosh) ===
  Huber (loss):              0.085934
  log_cosh:                  2.412238
  within_eps_0_005          : 0.010000
  within_eps_0_01           : 0.020000
  within_eps_0_02           : 0.032000
  within_eps_0_05           : 0.084000
  within_eps_0_1            : 0.148000
  AUTC[0.005–0.100]:         0.082895

=== Resultados L2 (120→20) ENS(25%AUTC+75%1/log_cosh) ===
  Huber (loss):              0.093748
  log_cosh:                  2.655495
  within_eps_0_005          : 0.006000
  within_eps_0_01           : 0.016000
  within_eps_0_02           : 0.024000
  within_eps_0_05           : 0.072000
  within_eps_0_1            : 0.146000
  AUTC[0.005–0.100]:         0.075211

=== Resultados L3 (120→20) ENS(25%AUTC+75%1/log